In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
! cp -av /content/drive/MyDrive/ThesisDataset/MedSet2/ ./dataset/

In [ ]:
import pandas as pd
import json
from markdown import markdown
from pathlib import Path
pd.set_option('max_colwidth',300)
from pprint import pprint

## Dataset for PLBart (testing)

In [ ]:
#train_py = pd.read_json('./dataset/SmallDataset/python/training.jsonl',lines=True)
#val_py = pd.read_json('./dataset/SmallDataset/python/validation.jsonl',lines=True)
#test_py = pd.read_json('./dataset/SmallDataset/python/testing.jsonl',lines=True)

train_java = pd.read_json('./dataset/java/training.jsonl',lines=True)
val_java = pd.read_json('./dataset/java/validation.jsonl',lines=True)
test_java = pd.read_json('./dataset/java/testing100.jsonl',lines=True)
"""
train_jscript = pd.read_json('./dataset/SmallDataset/javascript/training.jsonl',lines=True)
val_jscript = pd.read_json('./dataset/SmallDataset/javascript/validation.jsonl',lines=True)
test_jscript = pd.read_json('./dataset/SmallDataset/javascript/testing.jsonl',lines=True)

train_php = pd.read_json('./dataset/SmallDataset/php/training.jsonl',lines=True)
val_php = pd.read_json('./dataset/SmallDataset/php/validation.jsonl',lines=True)
test_php = pd.read_json('./dataset/SmallDataset/php/testing.jsonl',lines=True)

train_go = pd.read_json('./dataset/SmallDataset/go/training.jsonl',lines=True)
val_go = pd.read_json('./dataset/SmallDataset/go/validation.jsonl',lines=True)
test_go = pd.read_json('./dataset/SmallDataset/go/testing.jsonl',lines=True)

train_ruby = pd.read_json('./dataset/SmallDataset/ruby/training.jsonl',lines=True)
val_ruby = pd.read_json('./dataset/SmallDataset/ruby/validation.jsonl',lines=True)
test_ruby = pd.read_json('./dataset/SmallDataset/ruby/testing.jsonl',lines=True)"""

In [ ]:
test_java['code2'] = test_java.apply(lambda row: (''.join(row['code_tokens'][1:-1]).replace('\n',' ').replace("'",'')),axis=1)
test_java['code2'] = test_java.apply(lambda row: (''.join(row["code2"].strip().split()).replace(',',' ')),axis=1)

test_java['docstring2'] = test_java.apply(lambda row: (''.join(row['docstring_tokens'][1:-1]).replace('\n',' ').replace("'",'')),axis=1)
test_java['docstring2'] = test_java.apply(lambda row: (''.join(row["docstring2"].strip().split()).replace(',',' ')),axis=1)

test_java.head(2)

## Common Stuff

In [ ]:
! pip install transformers 

In [ ]:
! pip install tree_sitter

In [ ]:
! pip install sentencepiece

## GraphCodeBERT

### Seq2Seq

In [ ]:
# Copyright (c) Microsoft Corporation. 
# Licensed under the MIT license.

import torch
import torch.nn as nn
import torch
from torch.autograd import Variable
import copy
class Seq2Seq(nn.Module):
    """
        Build Seqence-to-Sequence.
        
        Parameters:

        * `encoder`- encoder of seq2seq model. e.g. roberta
        * `decoder`- decoder of seq2seq model. e.g. transformer
        * `config`- configuration of encoder model. 
        * `beam_size`- beam size for beam search. 
        * `max_length`- max length of target for beam search. 
        * `sos_id`- start of symbol ids in target for beam search.
        * `eos_id`- end of symbol ids in target for beam search. 
    """
    def __init__(self, encoder,decoder,config,beam_size=None,max_length=None,sos_id=None,eos_id=None):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder=decoder
        self.config=config
        self.register_buffer("bias", torch.tril(torch.ones(2048, 2048)))
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.lm_head = nn.Linear(config.hidden_size, config.vocab_size, bias=False)
        self.lsm = nn.LogSoftmax(dim=-1)
        self.tie_weights()
        
        self.beam_size=beam_size
        self.max_length=max_length
        self.sos_id=sos_id
        self.eos_id=eos_id
        
    def _tie_or_clone_weights(self, first_module, second_module):
        """ Tie or clone module weights depending of weither we are using TorchScript or not
        """
        if self.config.torchscript:
            first_module.weight = nn.Parameter(second_module.weight.clone())
        else:
            first_module.weight = second_module.weight
                  
    def tie_weights(self):
        """ Make sure we are sharing the input and output embeddings.
            Export to TorchScript can't handle parameter sharing so we are cloning them instead.
        """
        self._tie_or_clone_weights(self.lm_head,
                                   self.encoder.embeddings.word_embeddings)        
        
    def forward(self, source_ids,source_mask,position_idx,attn_mask,target_ids=None,target_mask=None,args=None):   
        #embedding
        nodes_mask=position_idx.eq(0)
        token_mask=position_idx.ge(2)        
        inputs_embeddings=self.encoder.embeddings.word_embeddings(source_ids)
        nodes_to_token_mask=nodes_mask[:,:,None]&token_mask[:,None,:]&attn_mask
        nodes_to_token_mask=nodes_to_token_mask/(nodes_to_token_mask.sum(-1)+1e-10)[:,:,None]
        avg_embeddings=torch.einsum("abc,acd->abd",nodes_to_token_mask,inputs_embeddings)
        inputs_embeddings=inputs_embeddings*(~nodes_mask)[:,:,None]+avg_embeddings*nodes_mask[:,:,None]  
        
        outputs = self.encoder(inputs_embeds=inputs_embeddings,attention_mask=attn_mask,position_ids=position_idx)
        encoder_output = outputs[0].permute([1,0,2]).contiguous()
        if target_ids is not None:  
            attn_mask=-1e4 *(1-self.bias[:target_ids.shape[1],:target_ids.shape[1]])
            tgt_embeddings = self.encoder.embeddings(target_ids).permute([1,0,2]).contiguous()
            out = self.decoder(tgt_embeddings,encoder_output,tgt_mask=attn_mask,memory_key_padding_mask=(1-source_mask).bool())
            hidden_states = torch.tanh(self.dense(out)).permute([1,0,2]).contiguous()
            lm_logits = self.lm_head(hidden_states)
            # Shift so that tokens < n predict n
            active_loss = target_mask[..., 1:].ne(0).view(-1) == 1
            shift_logits = lm_logits[..., :-1, :].contiguous()
            shift_labels = target_ids[..., 1:].contiguous()
            # Flatten the tokens
            loss_fct = nn.CrossEntropyLoss(ignore_index=-1)
            loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1))[active_loss],
                            shift_labels.view(-1)[active_loss])

            outputs = loss,loss*active_loss.sum(),active_loss.sum()
            return outputs
        else:
            #Predict
            my_logits=[] #new
            preds=[]       
            zero=torch.cuda.LongTensor(1).fill_(0)     
            for i in range(source_ids.shape[0]):
                context=encoder_output[:,i:i+1]
                context_mask=source_mask[i:i+1,:]
                beam = Beam(self.beam_size,self.sos_id,self.eos_id)
                input_ids=beam.getCurrentState()
                context=context.repeat(1, self.beam_size,1)
                context_mask=context_mask.repeat(self.beam_size,1)
                for _ in range(self.max_length): 
                    if beam.done():
                        break
                    attn_mask=-1e4 *(1-self.bias[:input_ids.shape[1],:input_ids.shape[1]])
                    tgt_embeddings = self.encoder.embeddings(input_ids).permute([1,0,2]).contiguous()
                    out = self.decoder(tgt_embeddings,context,tgt_mask=attn_mask,memory_key_padding_mask=(1-context_mask).bool())
                    out = torch.tanh(self.dense(out))
                    hidden_states=out.permute([1,0,2]).contiguous()[:,-1,:]
                    out = self.lsm(self.lm_head(hidden_states)).data
                    my_logits.append(out.clone()) #new  
                    final_logits = torch.cat(my_logits,0)# new
                    beam.advance(out)
                    input_ids.data.copy_(input_ids.data.index_select(0, beam.getCurrentOrigin()))
                    input_ids=torch.cat((input_ids,beam.getCurrentState()),-1)
                hyp= beam.getHyp(beam.getFinal())
                pred=beam.buildTargetTokens(hyp)[:self.beam_size]
                pred=[torch.cat([x.view(-1) for x in p]+[zero]*(self.max_length-len(p))).view(1,-1) for p in pred]

                preds.append(torch.cat(pred,0).unsqueeze(0))
            
            preds=torch.cat(preds,0)                
            return final_logits,preds   
        
        

class Beam(object):
    def __init__(self, size,sos,eos):
        self.size = size
        self.tt = torch.cuda
        # The score for each translation on the beam.
        self.scores = self.tt.FloatTensor(size).zero_()
        # The backpointers at each time-step.
        self.prevKs = []
        # The outputs at each time-step.
        self.nextYs = [self.tt.LongTensor(size)
                       .fill_(0)]
        self.nextYs[0][0] = sos
        # Has EOS topped the beam yet.
        self._eos = eos
        self.eosTop = False
        # Time and k pair for finished.
        self.finished = []

    def getCurrentState(self):
        "Get the outputs for the current timestep."
        batch = self.tt.LongTensor(self.nextYs[-1]).view(-1, 1)
        return batch

    def getCurrentOrigin(self):
        "Get the backpointers for the current timestep."
        return self.prevKs[-1]

    def advance(self, wordLk):
        """
        Given prob over words for every last beam `wordLk` and attention
        `attnOut`: Compute and update the beam search.

        Parameters:

        * `wordLk`- probs of advancing from the last step (K x words)
        * `attnOut`- attention at the last step

        Returns: True if beam search is complete.
        """
        numWords = wordLk.size(1)

        # Sum the previous scores.
        if len(self.prevKs) > 0:
            beamLk = wordLk + self.scores.unsqueeze(1).expand_as(wordLk)

            # Don't let EOS have children.
            for i in range(self.nextYs[-1].size(0)):
                if self.nextYs[-1][i] == self._eos:
                    beamLk[i] = -1e20
        else:
            beamLk = wordLk[0]
        flatBeamLk = beamLk.view(-1)
        bestScores, bestScoresId = flatBeamLk.topk(self.size, 0, True, True)

        self.scores = bestScores

        # bestScoresId is flattened beam x word array, so calculate which
        # word and beam each score came from
        prevK = bestScoresId // numWords
        self.prevKs.append(prevK)
        self.nextYs.append((bestScoresId - prevK * numWords))


        for i in range(self.nextYs[-1].size(0)):
            if self.nextYs[-1][i] == self._eos:
                s = self.scores[i]
                self.finished.append((s, len(self.nextYs) - 1, i))

        # End condition is when top-of-beam is EOS and no global score.
        if self.nextYs[-1][0] == self._eos:
            self.eosTop = True

    def done(self):
        return self.eosTop and len(self.finished) >=self.size

    def getFinal(self):
        if len(self.finished) == 0:
            self.finished.append((self.scores[0], len(self.nextYs) - 1, 0))
        self.finished.sort(key=lambda a: -a[0])
        if len(self.finished) != self.size:
            unfinished=[]
            for i in range(self.nextYs[-1].size(0)):
                if self.nextYs[-1][i] != self._eos:
                    s = self.scores[i]
                    unfinished.append((s, len(self.nextYs) - 1, i)) 
            unfinished.sort(key=lambda a: -a[0])
            self.finished+=unfinished[:self.size-len(self.finished)]
        return self.finished[:self.size]

    def getHyp(self, beam_res):
        """
        Walk back to construct the full hypothesis.
        """
        hyps=[]
        for _,timestep, k in beam_res:
            hyp = []
            for j in range(len(self.prevKs[:timestep]) - 1, -1, -1):
                hyp.append(self.nextYs[j+1][k])
                k = self.prevKs[j][k]
            hyps.append(hyp[::-1])
        return hyps
    
    def buildTargetTokens(self, preds):
        sentence=[]
        for pred in preds:
            tokens = []
            for tok in pred:
                if tok==self._eos:
                    break
                tokens.append(tok)
            sentence.append(tokens)
        return sentence

### Model load

In [ ]:
import torch
from transformers import AutoConfig, AutoModel, AutoTokenizer
MODEL_CLASSES = {'auto': (AutoConfig, AutoModel, AutoTokenizer)}
config_class, model_class, tokenizer_class = MODEL_CLASSES['auto']
config = AutoConfig.from_pretrained("microsoft/graphcodebert-base", vocab_size = 50005)#, hidden_dropout_prob=0.3)#,attention_probs_dropout_prob=0.2, hidden_dropout_prob=0.2)
tokenizer = AutoTokenizer.from_pretrained("uclanlp/plbart-base")

encoder = AutoModel.from_config(config)    
decoder_layer = nn.TransformerDecoderLayer(d_model=config.hidden_size, nhead=config.num_attention_heads)#, dropout=0.5)
decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
model=Seq2Seq(encoder=encoder,decoder=decoder,config=config,
              beam_size=1,max_length=128,
              sos_id=tokenizer.cls_token_id,eos_id=tokenizer.sep_token_id)
model.to("cuda")

In [ ]:
from transformers import AdamW

no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
      'weight_decay': 0.0},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
optimizer = AdamW(optimizer_grouped_parameters, lr=1e-4, eps=1.0)#,weight_decay=0.01)


checkpoint = torch.load("./drive/MyDrive/ThesisDataset/Models/GCB96_0305001_bleu_java")#,map_location='cpu'
model.load_state_dict(checkpoint)

In [ ]:
del checkpoint

### Input preparation for inference

In [ ]:
! git clone https://github.com/AntoniosMantzaris/CodeBERT.git

In [ ]:
import os 
os.chdir("CodeBERT/GraphCodeBERT/translation")
!ls

In [ ]:
from __future__ import absolute_import
import os
import sys
import pickle
import torch
import json
import random
import logging
import argparse
import numpy as np
from io import open
from itertools import cycle
import torch.nn as nn
from model import Seq2Seq
from tqdm import tqdm, trange
from bleu import _bleu
from torch.utils.data import DataLoader, Dataset, SequentialSampler, RandomSampler,TensorDataset
from torch.utils.data.distributed import DistributedSampler
from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                          AutoConfig, AutoModel, AutoTokenizer, PLBartTokenizer)
MODEL_CLASSES = {'auto': (AutoConfig, AutoModel, AutoTokenizer)}

logging.basicConfig(format = '%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
                    datefmt = '%m/%d/%Y %H:%M:%S',
                    level = logging.INFO)
from parser import DFG_python,DFG_java,DFG_ruby,DFG_go,DFG_php,DFG_javascript,DFG_csharp
from parser import (remove_comments_and_docstrings,
                   tree_to_token_index,
                   index_to_code_token,
                   tree_to_variable_index)
from tree_sitter import Language, Parser
dfg_function={
    'python':DFG_python,
    'java':DFG_java,
    'ruby':DFG_ruby,
    'go':DFG_go,
    'php':DFG_php,
    'javascript':DFG_javascript,
    'c_sharp':DFG_csharp,
}

source_lang = "java"
max_source_length = 320
max_target_length = 128


logger = logging.getLogger(__name__)
#load parsers
parsers={}
for lang in dfg_function:
    LANGUAGE = Language('parser/my-languages.so', lang)
    parser = Parser()
    parser.set_language(LANGUAGE)
    parser = [parser,dfg_function[lang]]
    parsers[lang]= parser

#remove comments, tokenize code and extract dataflow
def extract_dataflow(code, parser,lang):
    #remove comments
    try:
        code=remove_comments_and_docstrings(code,lang)
    except:
        pass
    #obtain dataflow
    if lang=="php":
        code="<?php"+code+"?>"
    try:
        tree = parser[0].parse(bytes(code,'utf8'))
        root_node = tree.root_node
        tokens_index=tree_to_token_index(root_node)
        code=code.split('\n')
        code_tokens=[index_to_code_token(x,code) for x in tokens_index]
        index_to_code={}
        for idx,(index,code) in enumerate(zip(tokens_index,code_tokens)):
            index_to_code[index]=(idx,code)
        try:
            DFG,_=parser[1](root_node,index_to_code,{})
        except:
            DFG=[]
        DFG=sorted(DFG,key=lambda x:x[1])
        indexs=set()
        for d in DFG:
            if len(d[-1])!=0:
                indexs.add(d[1])
            for x in d[-1]:
                indexs.add(x)
        new_DFG=[]
        for d in DFG:
            if d[1] in indexs:
                new_DFG.append(d)
        dfg=new_DFG
    except:
        dfg=[]
    return code_tokens,dfg


class Example(object):
    """A single training/test example."""
    def __init__(self,
                 idx,
                 source,
                 target,
                 ):
        self.idx = idx
        self.source = source
        self.target = target

def read_examples(filename):
    """Read examples from filename."""
    examples=[]
    with open(filename,encoding="utf-8") as f:
        for idx, line in enumerate(f):
            line=line.strip()
            js=json.loads(line)
            if 'idx' not in js:
                js['idx']=idx
            nl=''.join(js['docstring_tokens']).replace('\n',' ').replace("'",'')
            nl=''.join(nl.strip().split()).replace(',',' ')[1:-1]
            code = js["code"]
            examples.append(
                Example(
                        idx = idx,
                        source=code,
                        target = nl,
                        )
            )
    return examples


class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 example_id,
                 source_ids,
                 position_idx,
                 dfg_to_code,
                 dfg_to_dfg,
                 target_ids,
                 source_mask,
                 target_mask,

    ):
        self.example_id = example_id
        self.source_ids = source_ids
        self.position_idx = position_idx
        self.dfg_to_code = dfg_to_code
        self.dfg_to_dfg = dfg_to_dfg
        self.target_ids = target_ids
        self.source_mask = source_mask
        self.target_mask = target_mask


parsers={}
for lang in dfg_function:
    LANGUAGE = Language('parser/my-languages.so', lang)
    parser = Parser()
    parser.set_language(LANGUAGE)
    parser = [parser,dfg_function[lang]]
    parsers[lang]= parser

def convert_examples_to_features(examples, tokenizer,stage=None):
    features = []
    for example_index, example in enumerate(tqdm(examples,total=len(examples))):
        ##extract data flow
        code_tokens,dfg=extract_dataflow(example.source,
                                         parsers["c_sharp" if source_lang == "cs" else "java"],
                                         "c_sharp" if source_lang == "cs" else "java")
        code_tokens=[tokenizer.tokenize('@ '+x)[1:] if idx!=0 else tokenizer.tokenize(x) for idx,x in enumerate(code_tokens)]
        ori2cur_pos={}
        ori2cur_pos[-1]=(0,0)
        for i in range(len(code_tokens)):
            ori2cur_pos[i]=(ori2cur_pos[i-1][1],ori2cur_pos[i-1][1]+len(code_tokens[i]))
        code_tokens=[y for x in code_tokens for y in x]

        #truncating
        code_tokens=code_tokens[:max_source_length-3][:512-3]
        source_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
        source_ids =  tokenizer.convert_tokens_to_ids(source_tokens)
        position_idx = [i+tokenizer.pad_token_id + 1 for i in range(len(source_tokens))]
        dfg=dfg[:max_source_length-len(source_tokens)]
        source_tokens+=[x[0] for x in dfg]
        position_idx+=[0 for x in dfg]
        source_ids+=[tokenizer.unk_token_id for x in dfg]
        padding_length=max_source_length-len(source_ids)
        position_idx+=[tokenizer.pad_token_id]*padding_length
        source_ids+=[tokenizer.pad_token_id]*padding_length
        source_mask = [1] * (len(source_tokens))
        source_mask+=[0]*padding_length

        #reindex
        reverse_index={}
        for idx,x in enumerate(dfg):
            reverse_index[x[1]]=idx
        for idx,x in enumerate(dfg):
            dfg[idx]=x[:-1]+([reverse_index[i] for i in x[-1] if i in reverse_index],)
        dfg_to_dfg=[x[-1] for x in dfg]
        dfg_to_code=[ori2cur_pos[x[1]] for x in dfg]
        length=len([tokenizer.cls_token])
        dfg_to_code=[(x[0]+length,x[1]+length) for x in dfg_to_code]


        #target
        if stage=="test":
            target_tokens = tokenizer.tokenize("None")
        else:
            target_tokens = tokenizer.tokenize(example.target)[:max_target_length-2]
        target_tokens = [tokenizer.cls_token]+target_tokens+[tokenizer.sep_token]
        target_ids = tokenizer.convert_tokens_to_ids(target_tokens)
        target_mask = [1] *len(target_ids)
        padding_length = max_target_length - len(target_ids)
        target_ids+=[tokenizer.pad_token_id]*padding_length
        target_mask+=[0]*padding_length

        if example_index < 5:
            if stage=='train':
                logger.info("*** Example ***")
                
                logger.info("idx: {}".format(example.idx))

                logger.info("source_tokens: {}".format([x.replace('\u0120','_') for x in source_tokens]))
                logger.info("source_ids: {}".format(' '.join(map(str, source_ids))))
                logger.info("source_mask: {}".format(' '.join(map(str, source_mask))))

                logger.info("target_tokens: {}".format([x.replace('\u0120','_') for x in target_tokens]))
                logger.info("target_ids: {}".format(' '.join(map(str, target_ids))))
                logger.info("target_mask: {}".format(' '.join(map(str, target_mask))))

        features.append(
            InputFeatures(
                 example_index,
                 source_ids,
                 position_idx,
                 dfg_to_code,
                 dfg_to_dfg,
                 target_ids,
                 source_mask,
                 target_mask,
            )
        )
    return features

class TextDataset(Dataset):
    def __init__(self, examples):
        self.examples = examples

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        #calculate graph-guided masked function
        attn_mask=np.zeros((max_source_length,max_source_length),dtype=np.bool) #was self.max_length
        #calculate begin index of node and max length of input
        node_index=sum([i>1 for i in self.examples[item].position_idx])
        max_length=sum([i!=1 for i in self.examples[item].position_idx])
        #sequence can attend to sequence
        attn_mask[:node_index,:node_index]=True
        #special tokens attend to all tokens
        for idx,i in enumerate(self.examples[item].source_ids):
            if i in [0,2]:
                attn_mask[idx,:max_length]=True
        #nodes attend to code tokens that are identified from
        for idx,(a,b) in enumerate(self.examples[item].dfg_to_code):
            if a<node_index and b<node_index:
                attn_mask[idx+node_index,a:b]=True
                attn_mask[a:b,idx+node_index]=True
        #nodes attend to adjacent nodes
        for idx,nodes in enumerate(self.examples[item].dfg_to_dfg):
            for a in nodes:
                if a+node_index<len(self.examples[item].position_idx):
                    attn_mask[idx+node_index,a+node_index]=True

        return (torch.tensor(self.examples[item].source_ids),
                torch.tensor(self.examples[item].source_mask),
                torch.tensor(self.examples[item].position_idx),
                torch.tensor(attn_mask),
                torch.tensor(self.examples[item].target_ids),
                torch.tensor(self.examples[item].target_mask),)

In [ ]:
os.chdir("../../..")

### Inference

In [ ]:
test_filename="dataset/java/testing100.jsonl" 

logits = []
files=[]
files.append(test_filename)
for idx,file in enumerate(files):
  torch.cuda.empty_cache() 
  logger.info("Test file: {}".format(file))
  eval_examples = read_examples(file)
  eval_features = convert_examples_to_features(eval_examples, tokenizer, stage='test')
  eval_data = TextDataset(eval_features)

  eval_dataloader = DataLoader(eval_data,batch_size=1)
  p=[]
  for batch in tqdm(eval_dataloader,total=len(eval_dataloader)):
      batch = tuple(t.to("cuda") for t in batch)
      source_ids,source_mask,position_idx,att_mask,target_ids,target_mask = batch
      with torch.no_grad():
          logs, preds = model(source_ids,source_mask,position_idx,att_mask) #0:logits, 1:preds
          logits.append(logs.cpu().numpy())

In [ ]:
soft1 = []
pr = []
for ar in range(len(logits)):
  log_tens = torch.Tensor(logits[ar])
  softmaxed = torch.exp(log_tens)
  soft1.append(softmaxed)
  predictiones = torch.argmax(softmaxed, 1)
  pr.append(predictiones)

In [ ]:
temp = []
temp.append([0]*50005)
for i in range(len(soft1)):
  for length in range(128-len(soft1[i])):
    soft1[i] = torch.cat((soft1[i],torch.tensor(temp)),0)
soft1 #the softmax that will be used for the ensemble

In [ ]:
gcb_preds = tokenizer.batch_decode(pr, skip_special_tokens=True)
gcb_preds[:10]

In [ ]:
reference = []
for i in range(100):
  reference.append(test_java.docstring2.iloc[i])

In [ ]:
reference[:10]

## PLBart

In [ ]:
from transformers import PLBartForConditionalGeneration, PLBartTokenizer, PLBartConfig
config =PLBartConfig.from_pretrained("uclanlp/plbart-base")#,dropout=0.2, attention_dropout=0.2) 

tokenizer = PLBartTokenizer.from_pretrained("uclanlp/plbart-base")
modelo = PLBartForConditionalGeneration.from_pretrained("uclanlp/plbart-base")
#modelo = PLBartForConditionalGeneration(config)

In [ ]:
import torch

my_model = PLBartForConditionalGeneration(config)
checkpoint2 = torch.load('./drive/MyDrive/ThesisDataset/Models/PLBartMed2_last_java')
my_model.load_state_dict(checkpoint2)

In [ ]:
del checkpoint2

In [ ]:
def preprocessPL(data):
    return tokenizer(data['code2'], max_length = 320, padding=True, truncation=True, return_tensors="pt")

In [ ]:
from transformers import generation_utils

In [ ]:
import torch.nn as nn
sfm = nn.Softmax(dim=-1)

soft2 = []
final = []
reference = []
for i in range(100):
  input_tensor = preprocessPL(test_java.iloc[i])
  reference.append(test_java.docstring2.iloc[i])
  generation = my_model.generate(**input_tensor,return_dict_in_generate=True, output_scores=True)
  logi = generation.scores
  
  softs = [] #for logits (softmaxed)
  softi = []
  for tens in logi:
    s = sfm(tens)
    softs.append(s[0])
    softi.append(torch.argmax(s,-1).item())
  soft2.append(softs)
  final.append(softi)
final

In [ ]:
for i in range(len(soft2)):
  soft2[i]=torch.stack(soft2[i])

In [ ]:
temp = []
temp.append([0]*50005)
for i in range(len(soft2)):
  for length in range(128-len(soft2[i])):
    soft2[i] = torch.cat((soft2[i],torch.tensor(temp)),0)

In [ ]:
plbart_preds = []
for l in final:
  temp = []
  temp.append(l)
  t = torch.IntTensor(temp) #this way i can get the sentence and not the token sequence
  print(tokenizer.batch_decode(t, skip_special_tokens=True)[0])
  plbart_preds.append(tokenizer.batch_decode(t, skip_special_tokens=True)[0])

In [ ]:
plbart_preds[:10]

In [ ]:
reference[:10]

## BLEU-4

In [ ]:
#!/usr/bin/python

'''
This script was adapted from the original version by hieuhoang1972 which is part of MOSES. 
'''

# $Id: bleu.py 1307 2007-03-14 22:22:36Z hieuhoang1972 $

'''Provides:

cook_refs(refs, n=4): Transform a list of reference sentences as strings into a form usable by cook_test().
cook_test(test, refs, n=4): Transform a test sentence as a string (together with the cooked reference sentences) into a form usable by score_cooked().
score_cooked(alltest, n=4): Score a list of cooked test sentences.

score_set(s, testid, refids, n=4): Interface with dataset.py; calculate BLEU score of testid against refids.

The reason for breaking the BLEU computation into three phases cook_refs(), cook_test(), and score_cooked() is to allow the caller to calculate BLEU scores for multiple test sets as efficiently as possible.
'''

import sys, math, re, xml.sax.saxutils
import subprocess
import os

# Added to bypass NIST-style pre-processing of hyp and ref files -- wade
nonorm = 0

preserve_case = False
eff_ref_len = "shortest"

normalize1 = [
    ('<skipped>', ''),         # strip "skipped" tags
    (r'-\n', ''),              # strip end-of-line hyphenation and join lines
    (r'\n', ' '),              # join lines
#    (r'(\d)\s+(?=\d)', r'\1'), # join digits
]
normalize1 = [(re.compile(pattern), replace) for (pattern, replace) in normalize1]

normalize2 = [
    (r'([\{-\~\[-\` -\&\(-\+\:-\@\/])',r' \1 '), # tokenize punctuation. apostrophe is missing
    (r'([^0-9])([\.,])',r'\1 \2 '),              # tokenize period and comma unless preceded by a digit
    (r'([\.,])([^0-9])',r' \1 \2'),              # tokenize period and comma unless followed by a digit
    (r'([0-9])(-)',r'\1 \2 ')                    # tokenize dash when preceded by a digit
]
normalize2 = [(re.compile(pattern), replace) for (pattern, replace) in normalize2]

def normalize(s):
    '''Normalize and tokenize text. This is lifted from NIST mteval-v11a.pl.'''
    # Added to bypass NIST-style pre-processing of hyp and ref files -- wade
    if (nonorm):
        return s.split()
    if type(s) is not str:
        s = " ".join(s)
    # language-independent part:
    for (pattern, replace) in normalize1:
        s = re.sub(pattern, replace, s)
    s = xml.sax.saxutils.unescape(s, {'&quot;':'"'})
    # language-dependent part (assuming Western languages):
    s = " %s " % s
    if not preserve_case:
        s = s.lower()         # this might not be identical to the original
    for (pattern, replace) in normalize2:
        s = re.sub(pattern, replace, s)
    return s.split()

def count_ngrams(words, n=4):
    counts = {}
    for k in range(1,n+1):
        for i in range(len(words)-k+1):
            ngram = tuple(words[i:i+k])
            counts[ngram] = counts.get(ngram, 0)+1
    return counts

def cook_refs(refs, n=4):
    '''Takes a list of reference sentences for a single segment
    and returns an object that encapsulates everything that BLEU
    needs to know about them.'''
    
    refs = [normalize(ref) for ref in refs]
    maxcounts = {}
    for ref in refs:
        counts = count_ngrams(ref, n)
        for (ngram,count) in counts.items():
            maxcounts[ngram] = max(maxcounts.get(ngram,0), count)
    return ([len(ref) for ref in refs], maxcounts)

def cook_test(test, item, n=4):
    '''Takes a test sentence and returns an object that
    encapsulates everything that BLEU needs to know about it.'''
    (reflens, refmaxcounts)=item
    test = normalize(test)
    result = {}
    result["testlen"] = len(test)

    # Calculate effective reference sentence length.
    
    if eff_ref_len == "shortest":
        result["reflen"] = min(reflens)
    elif eff_ref_len == "average":
        result["reflen"] = float(sum(reflens))/len(reflens)
    elif eff_ref_len == "closest":
        min_diff = None
        for reflen in reflens:
            if min_diff is None or abs(reflen-len(test)) < min_diff:
                min_diff = abs(reflen-len(test))
                result['reflen'] = reflen

    result["guess"] = [max(len(test)-k+1,0) for k in range(1,n+1)]

    result['correct'] = [0]*n
    counts = count_ngrams(test, n)
    for (ngram, count) in counts.items():
        result["correct"][len(ngram)-1] += min(refmaxcounts.get(ngram,0), count)

    return result

def score_cooked(allcomps, n=4, ground=0, smooth=1):
    totalcomps = {'testlen':0, 'reflen':0, 'guess':[0]*n, 'correct':[0]*n}
    for comps in allcomps:
        for key in ['testlen','reflen']:
            totalcomps[key] += comps[key]
        for key in ['guess','correct']:
            for k in range(n):
                totalcomps[key][k] += comps[key][k]
    logbleu = 0.0
    all_bleus = []
    for k in range(n):
      correct = totalcomps['correct'][k]
      guess = totalcomps['guess'][k]
      addsmooth = 0
      if smooth == 1 and k > 0:
        addsmooth = 1
      logbleu += math.log(correct + addsmooth + sys.float_info.min)-math.log(guess + addsmooth+ sys.float_info.min)
      if guess == 0:
        all_bleus.append(-10000000)
      else:
        all_bleus.append(math.log(correct + sys.float_info.min)-math.log( guess ))

    logbleu /= float(n)
    all_bleus.insert(0, logbleu)

    brevPenalty = min(0,1-float(totalcomps['reflen'] + 1)/(totalcomps['testlen'] + 1))
    for i in range(len(all_bleus)):
      if i ==0:
        all_bleus[i] += brevPenalty
      all_bleus[i] = math.exp(all_bleus[i])
    return all_bleus

def bleu(refs,  candidate, ground=0, smooth=1):
    refs = cook_refs(refs)
    test = cook_test(candidate, refs)
    return score_cooked([test], ground=ground, smooth=smooth)

def splitPuncts(line):
  return ' '.join(re.findall(r"[\w]+|[^\s\w]", line))

def computeMaps(predictions, goldfile):
  predictionMap = {}
  goldMap = {}
  gf = goldfile #open(goldfile, 'r')

  id = 0
  for row in predictions:
    cols = row.strip().split('\t') 
    (rid, pred) = (id, cols[0])
    id+=1
    """
    if len(cols) == 1:
      (rid, pred) = (cols[0], '') 
    else:
      (rid, pred) = (cols[0], cols[1])"""
    predictionMap[rid] = [splitPuncts(pred.strip().lower())]
    
  id = 0
  for row in gf:
    cols = row.strip().split('\t')
    (rid, pred) = (id, cols[0])
    id+=1
    """if len(cols) == 1:
      (rid, pred) = (cols[0], '') 
    else:
      (rid, pred) = (cols[0], cols[1])""" 
    #goldMap[rid] = [splitPuncts(pred.strip().lower())]
    #(rid, pred) = row.split('\t') 
    if rid in predictionMap: # Only insert if the id exists for the method
      if rid not in goldMap:
        goldMap[rid] = []
      goldMap[rid].append(splitPuncts(pred.strip().lower()))
    #print(goldMap)
  sys.stderr.write('Total: ' + str(len(goldMap)) + '\n')
  return (goldMap, predictionMap)


#m1 is the reference map
#m2 is the prediction map
def bleuFromMaps(m1, m2):
  score = [0] * 5
  num = 0.0

  for key in m1:
    if key in m2:
      bl = bleu(m1[key], m2[key][0])
      score = [ score[i] + bl[i] for i in range(0, len(bl))]
      num += 1
  return [s * 100.0 / num for s in score]

"""if __name__ == '__main__':
  reference_file = sys.argv[1]
  predictions = []
  for row in sys.stdin:
    predictions.append(row)"""
(goldMap, predictionMap) = computeMaps(plbart_preds, reference) 
print(bleuFromMaps(goldMap, predictionMap)[0])



## Ensemble

In [ ]:
"""del gcb_preds
del plbart_preds
del temp
del input_tensor
del generation
del final
del p
del pr
del model
del my_model"""

### Sentence union

In [ ]:
union_pred = []
for i in range(100):
  union_pred.append(plbart_preds[i]+" " +gcb_preds[i])

In [ ]:
union_pred[:10]

### Mean

In [ ]:
def mean_ens():
  avg =[]
  for idx in range(100):
    sum = torch.add(soft1[idx], soft2[idx], alpha=1)  #*alpha the softmaxed2/ just for this testing
    avg.append(torch.div(sum,2))

  preds_avg = []
  for i in range(len(avg)):
    preds_avg.append(torch.argmax(torch.tensor(avg[i]), -1).tolist())

  mean_preds = []
  for l in preds_avg:
    temp = []
    temp.append(l)
    t = torch.IntTensor(temp) #this way i can get the sentence and not the token sequence
    mean_preds.append(tokenizer.batch_decode(t, skip_special_tokens=True)[0])

  return mean_preds   

In [ ]:
preds1 = mean_ens()

In [ ]:
preds1[:10]

### Max

In [ ]:
def max_ens():
  maximum =[]
  for idx in range(100):
    maximum.append(torch.max(soft1[idx], soft2[idx]))
  preds_max = []
  
  for i in range(len(maximum)):
    preds_max.append(torch.argmax(torch.tensor(maximum[i]), -1).tolist())
   
  max_preds = []
  for l in preds_max:
    temp = []
    temp.append(l)
    t = torch.IntTensor(temp) #this way i can get the sentence and not the token sequence
    max_preds.append(tokenizer.batch_decode(t, skip_special_tokens=True)[0])

  return max_preds

In [ ]:
preds2 = max_ens()

In [ ]:
preds2[:10]

### Weighted mean

In [ ]:
def weighted_mean(weightGCB, weightPLBart):
  avg_weighted =[]
  for idx in range(100):
    sum_w = torch.add(weightGCB*soft1[idx], weightPLBart*soft2[idx], alpha=1)  #*alpha the softmaxed2/ just for this testing
    avg_weighted.append(torch.div(sum_w,weightGCB+weightPLBart))

  preds_weighted_avg = []
  for i in range(len(avg_weighted)):
    preds_weighted_avg.append(torch.argmax(torch.tensor(avg_weighted[i]), -1).tolist())

  weighted_mean_preds = []
  for l in preds_weighted_avg:
    temp = []
    temp.append(l)
    t = torch.IntTensor(temp) #this way i can get the sentence and not the token sequence
    weighted_mean_preds.append(tokenizer.batch_decode(t, skip_special_tokens=True)[0])

  return weighted_mean_preds

In [ ]:
preds3 = weighted_mean(1,2)

In [ ]:
preds3[:10]

In [ ]:
reference[:10]

### Weighted max

In [ ]:
def weighted_max(weightGCB,weightPLBart):
  w_maximum =[]
  for idx in range(100):
    w_maximum.append(torch.max(weightGCB*soft1[idx], weightPLBart*soft2[idx]))  

  preds_max_w = []
  for i in range(len(w_maximum)):
    preds_max_w.append(torch.argmax(torch.tensor(w_maximum[i]), -1).tolist())

  weighted_max_preds = []
  for l in preds_max_w:
    temp = []
    temp.append(l)
    t = torch.IntTensor(temp) #this way i can get the sentence and not the token sequence
    weighted_max_preds.append(tokenizer.batch_decode(t, skip_special_tokens=True)[0])

  return weighted_max_preds

In [ ]:
preds4 = weighted_max(8,1)

In [ ]:
preds4[:10]

In [ ]:
reference[:10]

### DoubleEnsemble (weighted mean(weighted_max,PLBart))

In [ ]:
def doubleEns(weightMaxGCB, weightMaxPLBart, weightEns, weightPLBart):
  w_maximum =[]
  for idx in range(100):
    w_maximum.append(torch.max(weightMaxGCB*soft1[idx], weightMaxPLBart*soft2[idx]))

  avg_weighted =[]
  for idx in range(100):
    sum_w = torch.add(weightEns*w_maximum[idx], weightPLBart*soft2[idx], alpha=1)  #*alpha the softmaxed2/ just for this testing
    avg_weighted.append(torch.div(sum_w,weightEns+weightPLBart))

  preds_weighted_avg2 = []
  for i in range(len(avg_weighted)):
    preds_weighted_avg2.append(torch.argmax(torch.tensor(avg_weighted[i]), -1).tolist())

  weighted_mean_preds2 = []
  for l in preds_weighted_avg2:
    temp = []
    temp.append(l)
    t = torch.IntTensor(temp) #this way i can get the sentence and not the token sequence
    weighted_mean_preds2.append(tokenizer.batch_decode(t, skip_special_tokens=True)[0])

  return weighted_mean_preds2     

In [ ]:
preds5 = doubleEns(8,1,1,15)

In [ ]:
preds5[:10]

In [ ]:
reference[:10]

## BLEU-4 (ensemble/same)

In [ ]:
#!/usr/bin/python

'''
This script was adapted from the original version by hieuhoang1972 which is part of MOSES. 
'''

# $Id: bleu.py 1307 2007-03-14 22:22:36Z hieuhoang1972 $

'''Provides:

cook_refs(refs, n=4): Transform a list of reference sentences as strings into a form usable by cook_test().
cook_test(test, refs, n=4): Transform a test sentence as a string (together with the cooked reference sentences) into a form usable by score_cooked().
score_cooked(alltest, n=4): Score a list of cooked test sentences.

score_set(s, testid, refids, n=4): Interface with dataset.py; calculate BLEU score of testid against refids.

The reason for breaking the BLEU computation into three phases cook_refs(), cook_test(), and score_cooked() is to allow the caller to calculate BLEU scores for multiple test sets as efficiently as possible.
'''

import sys, math, re, xml.sax.saxutils
import subprocess
import os

# Added to bypass NIST-style pre-processing of hyp and ref files -- wade
nonorm = 0

preserve_case = False
eff_ref_len = "shortest"

normalize1 = [
    ('<skipped>', ''),         # strip "skipped" tags
    (r'-\n', ''),              # strip end-of-line hyphenation and join lines
    (r'\n', ' '),              # join lines
#    (r'(\d)\s+(?=\d)', r'\1'), # join digits
]
normalize1 = [(re.compile(pattern), replace) for (pattern, replace) in normalize1]

normalize2 = [
    (r'([\{-\~\[-\` -\&\(-\+\:-\@\/])',r' \1 '), # tokenize punctuation. apostrophe is missing
    (r'([^0-9])([\.,])',r'\1 \2 '),              # tokenize period and comma unless preceded by a digit
    (r'([\.,])([^0-9])',r' \1 \2'),              # tokenize period and comma unless followed by a digit
    (r'([0-9])(-)',r'\1 \2 ')                    # tokenize dash when preceded by a digit
]
normalize2 = [(re.compile(pattern), replace) for (pattern, replace) in normalize2]

def normalize(s):
    '''Normalize and tokenize text. This is lifted from NIST mteval-v11a.pl.'''
    # Added to bypass NIST-style pre-processing of hyp and ref files -- wade
    if (nonorm):
        return s.split()
    if type(s) is not str:
        s = " ".join(s)
    # language-independent part:
    for (pattern, replace) in normalize1:
        s = re.sub(pattern, replace, s)
    s = xml.sax.saxutils.unescape(s, {'&quot;':'"'})
    # language-dependent part (assuming Western languages):
    s = " %s " % s
    if not preserve_case:
        s = s.lower()         # this might not be identical to the original
    for (pattern, replace) in normalize2:
        s = re.sub(pattern, replace, s)
    return s.split()

def count_ngrams(words, n=4):
    counts = {}
    for k in range(1,n+1):
        for i in range(len(words)-k+1):
            ngram = tuple(words[i:i+k])
            counts[ngram] = counts.get(ngram, 0)+1
    return counts

def cook_refs(refs, n=4):
    '''Takes a list of reference sentences for a single segment
    and returns an object that encapsulates everything that BLEU
    needs to know about them.'''
    
    refs = [normalize(ref) for ref in refs]
    maxcounts = {}
    for ref in refs:
        counts = count_ngrams(ref, n)
        for (ngram,count) in counts.items():
            maxcounts[ngram] = max(maxcounts.get(ngram,0), count)
    return ([len(ref) for ref in refs], maxcounts)

def cook_test(test, item, n=4):
    '''Takes a test sentence and returns an object that
    encapsulates everything that BLEU needs to know about it.'''
    (reflens, refmaxcounts)=item
    test = normalize(test)
    result = {}
    result["testlen"] = len(test)

    # Calculate effective reference sentence length.
    
    if eff_ref_len == "shortest":
        result["reflen"] = min(reflens)
    elif eff_ref_len == "average":
        result["reflen"] = float(sum(reflens))/len(reflens)
    elif eff_ref_len == "closest":
        min_diff = None
        for reflen in reflens:
            if min_diff is None or abs(reflen-len(test)) < min_diff:
                min_diff = abs(reflen-len(test))
                result['reflen'] = reflen

    result["guess"] = [max(len(test)-k+1,0) for k in range(1,n+1)]

    result['correct'] = [0]*n
    counts = count_ngrams(test, n)
    for (ngram, count) in counts.items():
        result["correct"][len(ngram)-1] += min(refmaxcounts.get(ngram,0), count)

    return result

def score_cooked(allcomps, n=4, ground=0, smooth=1):
    totalcomps = {'testlen':0, 'reflen':0, 'guess':[0]*n, 'correct':[0]*n}
    for comps in allcomps:
        for key in ['testlen','reflen']:
            totalcomps[key] += comps[key]
        for key in ['guess','correct']:
            for k in range(n):
                totalcomps[key][k] += comps[key][k]
    logbleu = 0.0
    all_bleus = []
    for k in range(n):
      correct = totalcomps['correct'][k]
      guess = totalcomps['guess'][k]
      addsmooth = 0
      if smooth == 1 and k > 0:
        addsmooth = 1
      logbleu += math.log(correct + addsmooth + sys.float_info.min)-math.log(guess + addsmooth+ sys.float_info.min)
      if guess == 0:
        all_bleus.append(-10000000)
      else:
        all_bleus.append(math.log(correct + sys.float_info.min)-math.log( guess ))

    logbleu /= float(n)
    all_bleus.insert(0, logbleu)

    brevPenalty = min(0,1-float(totalcomps['reflen'] + 1)/(totalcomps['testlen'] + 1))
    for i in range(len(all_bleus)):
      if i ==0:
        all_bleus[i] += brevPenalty
      all_bleus[i] = math.exp(all_bleus[i])
    return all_bleus

def bleu(refs,  candidate, ground=0, smooth=1):
    refs = cook_refs(refs)
    test = cook_test(candidate, refs)
    return score_cooked([test], ground=ground, smooth=smooth)

def splitPuncts(line):
  return ' '.join(re.findall(r"[\w]+|[^\s\w]", line))

def computeMaps(predictions, goldfile):
  predictionMap = {}
  goldMap = {}
  gf = goldfile #open(goldfile, 'r')

  id = 0
  for row in predictions:
    cols = row.strip().split('\t') 
    (rid, pred) = (id, cols[0])
    id+=1
    """
    if len(cols) == 1:
      (rid, pred) = (cols[0], '') 
    else:
      (rid, pred) = (cols[0], cols[1])"""
    predictionMap[rid] = [splitPuncts(pred.strip().lower())]
    
  id = 0
  for row in gf:
    cols = row.strip().split('\t')
    (rid, pred) = (id, cols[0])
    id+=1
    """if len(cols) == 1:
      (rid, pred) = (cols[0], '') 
    else:
      (rid, pred) = (cols[0], cols[1])""" 
    #goldMap[rid] = [splitPuncts(pred.strip().lower())]
    #(rid, pred) = row.split('\t') 
    if rid in predictionMap: # Only insert if the id exists for the method
      if rid not in goldMap:
        goldMap[rid] = []
      goldMap[rid].append(splitPuncts(pred.strip().lower()))
    #print(goldMap)
  sys.stderr.write('Total: ' + str(len(goldMap)) + '\n')
  return (goldMap, predictionMap)


#m1 is the reference map
#m2 is the prediction map
def bleuFromMaps(m1, m2):
  score = [0] * 5
  num = 0.0

  for key in m1:
    if key in m2:
      bl = bleu(m1[key], m2[key][0])
      score = [ score[i] + bl[i] for i in range(0, len(bl))]
      num += 1
  return [s * 100.0 / num for s in score]

"""if __name__ == '__main__':
  reference_file = sys.argv[1]
  predictions = []
  for row in sys.stdin:
    predictions.append(row)"""
(goldMap, predictionMap) = computeMaps(preds5, reference) 
print(bleuFromMaps(goldMap, predictionMap)[0])

